### Read Username and PW from local

In [ ]:
import pandas as pd
import os

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys

import time
import datetime
from collections import defaultdict
import re

In [ ]:
studio_pref=[4,3,2,1]
instructors = ['JEAN','JE','TIFFANY','NIC G','HYEKEL','VANESSA','LYDIA','JELINA']

#Parameters for zouk onlyere
zouk_para = {'pref_time_wkday': ['7:15 PM','8:30 PM', '6:00 PM'],
'pref_time_wkend': ['10:45 AM','9:30 AM', '12:00 PM', '1:15 PM', '2:30 PM', '3:45 PM'],
'pref_bikes_no': [40,41,42,43,44,45,46,47,48,49,50]}

# #Parameters for Milennia only
milennia_para = {'pref_time_wkday': ['7:00 PM','8:15 PM'],
'pref_time_wkend': ['10:45 AM','9:30 AM', '12:00 PM', '1:15 PM', '2:30 PM', '3:45 PM'],
'pref_bikes_no': []}

#Parameters for Downtown Gallery only
downtown_para = {'pref_time_wkday': ['6:45 PM','7:45 PM'],
'pref_time_wkend': ['10:15 AM','9:00 AM', '11:30 AM', '12:45 PM', '2:00 PM', '3:15 PM'],
'pref_bikes_no': []}

#Parameters for Centrepoint only
centrepoint_para = {'pref_time_wkday': ['6:45 PM','7:45 PM'],
'pref_time_wkend': ['10:15 AM','9:00 AM', '11:30 AM', '12:45 PM', '2:00 PM', '3:15 PM'],
'pref_bikes_no': []}

In [ ]:
#Expected timeslot to book

ser_instructor = input("Key in instructor name in caps from following list: [JEAN, JE, TIFFANY, NIC G, HYEKEL, VANESSA, LYDIA, JELINA] ")
ser_timeslot = input("Key in timeslot in format (HH:MM AM/PM): ")
ser_gym = input("Key in gym: ")

ser_parameters=[ser_instructor,ser_timeslot,ser_gym]
print(ser_parameters)

In [ ]:
def studio_para(x):
    if x ==4:
        return zouk_para
    elif x == 3:
        return milennia_para
    elif x ==2:
        return downtown_para
    elif x==1:
        return centrepoint_para

dict_pref = {studio: studio_para(studio) for studio in studio_pref}

dict_pref

In [ ]:
def read_file():
    filepath = os.path.join(os.getcwd(),'desktop','abscycle','serenepw.csv')
    file = pd.read_csv(filepath,header=None)
    return file
    
file = read_file()
user = file.loc[0,0]
pw = file.loc[0,1]

In [ ]:
#Mon=0, Sun=6
def wkdaycheck():
    wknow = datetime.datetime.today().weekday()
    if wknow < 5:
        return 'weekday'
    else:
        return 'weekend'
    
wkdaycheck()

### Navigate website homepage and login

In [ ]:
#Instantiate driver object
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)
url = 'https://www.absolutecyclesingapore.com/'

In [ ]:
driver.get(url)

#Traverse to login page
WebDriverWait(driver, 3).until(\
EC.presence_of_element_located((By.XPATH, 
                                 ".//a[contains(text(), 'LOG IN')]"))).click()

In [ ]:
def homepage(driver):

    #switch frame as elements are on different frame
    driver.refresh()
    try:
        driver.switch_to.frame(0)
    except:
        return ("Error in switching frame! Bot failed!")

    #key in username and password
    user_element = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='username']")))
    user_element.send_keys(user)

    pw_element = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='password']")))
    pw_element.send_keys(pw)

    #login to account
    WebDriverWait(driver, 3).until(\
    EC.presence_of_element_located((By.XPATH, 
                                     ".//button[contains(text(), 'Sign In')]"))).click()

In [ ]:
homepage(driver)

### Navigate to studios page

In [ ]:
def nav_studios(driver,studio_pref):
    driver.refresh()

    xpth = "//div[@class='container header-menu-nav-item header-menu-nav-item--external']/a"

    WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, xpth)))
    elems = driver.find_elements_by_xpath(xpth)

    for elem in elems:
        href_list=[elem.get_attribute('href') for elem in elems]
        
    new_href_list=[]
    for studio in studio_pref:
        for href in href_list:
            if href.endswith("schedule/site/" + str(studio)):
                new_href_list.append(href)
        
    return new_href_list

In [ ]:
new_href_list = nav_studios(driver,studio_pref)
new_href_list

### Navigate in Individual Studios to press next week slots

In [ ]:
#Go to next week (click) in individual studio
def next_week_slots(driver):
    
    driver.refresh()
    driver.switch_to.frame(0)

    xpth1 = "//li[@class='next']/a"

    WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, xpth1)))
    next_elem = driver.find_element_by_xpath(xpth1)
    
    next_week_slots = next_elem.get_attribute('href')
    
    return next_week_slots

In [ ]:
#Test out 1 studio - Zouk first
for href in new_href_list:
    driver.get(href)
    break

In [ ]:
#Production situation
for href in new_href_list:
    
    #pull out studio parameters
    studio_number = int(href[-1])
    pref=dict_pref[studio_number]
    
    pref_time_wkday = pref['pref_time_wkday']
    pref_time_wkend = pref['pref_time_wkend']
    pref_bikes_no = pref['pref_bikes_no']
    
    driver.get(href)

    next_week_slots1 = next_week_slots(driver)
    
    driver.get(next_week_slots1)
    
    output=avail_slots(driver)
    
    output_dicts = slot_dict(driver,output[0])
    
    try:
        elem_dict = output_dicts[0]
        href_dict = output_dicts[1]

        print(f"elem_dict is {elem_dict}")
        print(f"href_dict is {href_dict}")
    
    except TypeError as e:
        elem_dict={}
        href_dict={}
        print(f"{e}: No available classes open!")
        
    try:
        pref_slot(driver,elem_dict,href_dict,wkdaycheck())
    except:
        continue

    

In [ ]:
next_week_slots = next_week_slots(driver)

In [ ]:
driver.get(next_week_slots)

### Navigate to find available slots next week

In [ ]:
#select day 0 (exactly 7 days next week) at inidividual studio and get avail/waitlist
def avail_slots(driver):
    
    driver.refresh()
    driver.switch_to.frame(0)

    xpth_slots= "//td[@class='day0']"

    WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, xpth_slots)))
    next_elems = driver.find_elements_by_xpath("//td[@class='day0']//child::div")
    
    class_ids = {elem.get_attribute('data-classid'):elem.get_attribute('class') for elem in next_elems}
    
    avail_class_ids = [k for k,v in class_ids.items() if v == 'scheduleBlock bookable' ]
    waitlist_class_ids = [k for k,v in class_ids.items() if v == 'scheduleBlock classfull' ]
    
    return avail_class_ids,waitlist_class_ids

In [ ]:
output = avail_slots(driver)
avail_class_ids = output[0]
avail_class_ids

In [ ]:
#check individual bookable Ids are within parameters and click in
def slot_dict(driver,avail_class_ids):
    
    elem_dict=defaultdict(list)
    href_dict=defaultdict(list)
    
    driver.refresh()
    driver.switch_to.frame(0)
    
    for slot in avail_class_ids:
        
        xpth_slots1 = "//div[@data-classid=" + slot + "]"
        xpth_slots1_child_span = xpth_slots1 + '//child::span'
        xpth_slots1_child_href = xpth_slots1 + '//child::a'
        WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, xpth_slots1)))
        
        next_elems = driver.find_elements_by_xpath(xpth_slots1_child_span)
        elem_dict[slot].append(next_elems)
        
        WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, xpth_slots1_child_href)))
        href_elems = driver.find_elements_by_xpath(xpth_slots1_child_href)
        href_dict[slot].append(href_elems)
        
    return elem_dict,href_dict

In [ ]:
output_dicts = slot_dict(driver,avail_class_ids)
try:
    elem_dict = output_dicts[0]
    href_dict = output_dicts[1]
    
    print(elem_dict)
    print()
    print(href_dict)
    
except TypeError as e:
    elem_dict={}
    href_dict={}
    print(f"{e}: No available classes open!")


In [ ]:
#Define book bikes function inside bike page
def book_bikes(driver):
#     driver.switch_to.frame(0)
    xpth_slots2 = "//div[@id='spotwrapper']//child::a"
    WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, xpth_slots2)))

    next_elems2 = driver.find_elements_by_xpath(xpth_slots2)
    
    for elem in next_elems2:
        bike_no = re.findall('[0-9]+',elem.get_attribute('id'))[0]
    
#         if int(bike_no) not in zouk_para['pref_bikes_no']
        if int(bike_no) not in pref_bikes_no:
            
            print(elem.get_attribute('href'))

            driver.get(elem.get_attribute('href'))
            print(f"No {bike_no} bike booked!")
            break
        
        else:
            print(f"No bikes available!")

In [ ]:
#check whether preferred instruc and timeslots available, and get href and click in and book bikes
def pref_slot(driver,elem_dict,href_dict,is_weekday):
    
    instruct=False
    time=False
    
    if is_weekday == 'weekday':
#         time_slots = zouk_para['pref_time_wkday']
        time_slots = pref_time_wkday
    elif is_weekday == 'weekend':
#         time_slots = zouk_para['pref_time_wkend']
        time_slots = pref_time_wkend
    
    if elem_dict=={} or href_dict=={}:
        print('No slots available!')
        print()
        
    else:
    
        for slot,elems in elem_dict.items():
            for element in elems[0]:

                if element.text.rstrip() in instructors:
#                     print(f"Yes, instructor {element.text} slot found!")
                    instruct=element.text

                if element.text.rstrip('45 MIN').rstrip() in time_slots:
#                     print((f"Yes, time slot @ {element.text} found!"))
                    time=element.text

            if instruct and time:
                to_click = href_dict[slot][0][0].get_attribute('href')
                print(f"{instruct} at {time} found!")

#                     driver.refresh()
#                     driver.switch_to.frame(0)
#                     time.sleep(2)

                driver.get(to_click)
                
                try:
                    book_bikes(driver)
                    print('Successfully went to bikes page and booked!')
                    break
                
                except:
                    print("Went to login page...")
                    homepage(driver)
                    print("Relogined again!")
                    
                    book_bikes(driver)
                    print("Successfully went to bikes page and booked after relogin!")
                    break
                    

In [ ]:
instructors=['AMINAH']
time_slots=['10:30 AM','9:30 AM']

In [ ]:
intruct=False
time=False

for slot,elems in elem_dict.items():
    print(slot)
    for element in elems[0]:
        if element.text.rstrip() in instructors:
            print(element.text.rstrip())
            instruct=element.text.rstrip()
        
        if element.text.rstrip('45 MIN').rstrip() in time_slots:
            print(element.text.rstrip('45 MIN').rstrip())
            time=element.text.rstrip('45 MIN').rstrip()
            
    if instruct and time:
        print(f"{instruct} at {time} found!")
        to_click = href_dict[slot][0][0].get_attribute('href')
        print(to_click)
        instruct=False
        time=False
        
    print()

In [ ]:
href_dict

In [ ]:
pref_slot(driver,elem_dict,href_dict,wkdaycheck())

In [ ]:
for slot,elems in elem_dict.items():
    print(slot)
    for element in elems[0]:
        print(element.text)
        abc= element.text.rstrip('45 MIN').rstrip()
        print(abc)
        if abc in time_slots:
            print("YES")
        print('a')
    print('next')